In [1]:
# How bad is it to import all of numpy and torch?

import numpy as np
import matplotlib.pyplot as plt
import torch
import sys
sys.path.append('../src_variable')

import network as nw 
import road as rd 
import junction as jn 
import traffic_lights as tl 
import generate_kvadraturen as gk
import bus as bs

In [2]:
import cProfile
import pstats

In [3]:
network = gk.generate_kvadraturen_small(100.0)

In [6]:
ids = ["v_strand_1", "v_strand_2", "v_strand_3", "v_strand_4", "v_strand_5", "v_strand_6", "v_strand_7",
    "tollbod_1", "tollbod_2", "tollbod_3", "tollbod_4", "tollbod_5", "tollbod_6",
    "elvegata", "lundsbro"]
bus19 = bs.Bus([], [], [], network)

TypeError: Bus.__init__() takes 4 positional arguments but 5 were given

In [4]:
history_of_network, queues, bus_times = network.solve_cons_law()

c:\Users\torje\OneDrive - NTNU\Dokumenter\Semester10\kristiansand\notebooks\../src_variable\network.py:268: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dt = torch.tensor(controlpoint - t)


In [6]:
print(bus_times)

{}


In [23]:
min_x = 1e10
max_x = -1e10
min_y = 1e10
max_y = -1e10

for road in network.roads:
    left = road.left_pos
    right = road.right_pos
    min_x = min(min_x, left[0], right[0])
    max_x = max(max_x, left[0], right[0])
    min_y = min(min_y, left[1], right[1])
    max_y = max(max_y, left[1], right[1])

In [33]:
a = [[[None, None], [None, None]]] * 5
b = [4] * 5



TypeError: Value after * must be an iterable, not int

In [26]:
a_x = 4 / (max_x - min_x)
b_x = 2 -  4 * max_x / (max_x - min_x)

a_y = 4 / (max_y - min_y)
b_y = 2 -  4 * max_y / (max_y - min_y)

for i, road in enumerate(network.roads):
    left = road.left_pos
    right = road.right_pos
    left = [a_x * left[0] + b_x, a_y * left[1] + b_y]
    right = [a_x * right[0] + b_x, a_y * right[1] + b_y]
    print(left)
    print(right)
    print()

    if i == 10:
        break

[-1.5, -2.0]
[-1.5, -1.5555555555555556]

[-1.5, -1.5555555555555556]
[-1.5, -1.1111111111111112]

[-1.5, -1.1111111111111112]
[-1.5, -0.6666666666666667]

[-1.5, -0.6666666666666667]
[-1.5, -0.22222222222222232]

[-1.5, -0.22222222222222232]
[-1.5, 0.22222222222222232]

[-1.5, 0.22222222222222232]
[-1.5, 0.6666666666666665]

[-1.5, 0.6666666666666665]
[-1.5, 1.1111111111111107]

[-1.5, 1.1111111111111107]
[-1.5, 1.5555555555555554]

[-1.5, -1.5555555555555556]
[-1.5, -2.0]

[-1.5, -1.1111111111111112]
[-1.5, -1.5555555555555556]

[-1.5, -0.6666666666666667]
[-1.5, -1.1111111111111112]



In [9]:
def run():
    network.solve_cons_law()

cProfile.run('run()', 'restats')

c:\Users\torje\OneDrive - NTNU\Dokumenter\Semester10\kristiansand\notebooks\../src_variable\network.py:260: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dt = torch.tensor(controlpoint - t)


Trying to generate the junctions and roads in "kvadraturen" in Kristiansand.
This road section is approximately a 10x9 rectangular grid with lengths around 100m and 50m's.
For now start by assuming that all roads are two directional, so that there needs to be added two roads for each section.
Later change this by setting some roads to be one directional.

At some point, store information on roads in json file...

In [2]:
class DummyRoad:
    start_x = 0
    end_x = 0
    start_y = 0
    end_y = 0

    def __init__(self, start_x, start_y, end_x, end_y):
        self.start_x = start_x
        self.end_x = end_x
        self.start_y = start_y
        self.end_y = end_y

class DummyJunction:
    x_pos = 0
    y_pos = 0
    in_roads = []
    out_roads = []

    def __init__(self, x_pos, y_pos, in_roads, out_roads):
        self.x_pos = x_pos
        self.y_pos = y_pos
        self.in_roads = in_roads
        self.out_roads = out_roads

Idea of algorithm: Go through every junction of "Kvadraturen". 

For the first version of the algorithm, all junctions are assumed to have two roads (one incoming and one outgoing) in all four directions.
Interior roads will connect to two different junctions, and boundary roads will only connect to only one junction


Idea of first version of the algorithm:

n horizontal roads, m vertical roads

When travelling to the right or downwards, we will say we are moving forwards.
When travelling to the left or upwards, we will say we are moving backwards.

Go through each junction in the grid (ordered in a nxm matrix).

For the first column of the grid matrix the incoming/outgoing roads to the left of the junction will be created.
For the first row of the grid matrix the incoming/outgoing roads above the junction will be created.
For all junctions the incoming/outgoing roads to the left and above the junction will be created.

The index of the road in the matrices will be related to the index of the junction that created it.

The eight roads related to each junction are then created, and the junction can be created.

In [3]:
# Go through all junctions of grid and add horizontal and vertical roads going into and out of the junctions
N = 4 # Horizontal roads
M = 3 # Vertical roads

# Storing the roads in MxN matrices
# +1 since there are boundary roads on both sides of the grid
fwd_hor = [[None for _ in range(M+1)] for _ in range(N)]
fwd_ver = [[None for _ in range(M)] for _ in range(N+1)]
bwd_hor = [[None for _ in range(M+1)] for _ in range(N)]
bwd_ver = [[None for _ in range(M)] for _ in range(N+1)]

# Creating the junctions
junctions = [[None for _ in range(M)] for _ in range(N)]

for i in range(N): 
    for j in range(M):
        # Node at position (i,j)
        
        if i == 0:
            # The first row of nodes
            if j == 0:
                # The first corner of grid
                # Need to create all 8 roads
                
                # To the left
                fwd_hor[0][0] = DummyRoad(-1, 0, 0, 0)
                bwd_hor[0][0] = DummyRoad(0, 0, -1, 0)

                # Above
                fwd_ver[0][0] = DummyRoad(0, -1, 0, 0)
                bwd_ver[0][0] = DummyRoad(0, 0, 0, -1)

                # To the right
                fwd_hor[0][1] = DummyRoad(0, 0, 1, 0)
                bwd_hor[0][1] = DummyRoad(1, 0, 0, 0)
            
                # Below
                fwd_ver[1][0] = DummyRoad(0, 0, 0, 1)
                bwd_ver[1][0] = DummyRoad(0, 1, 0, 0)

                # Add to junction 
                junctions[0][0] = DummyJunction(0, 0, [fwd_hor[0][0], fwd_ver[0][0], bwd_hor[0][1], bwd_ver[1][0]], 
                                                [bwd_hor[0][0], bwd_ver[0][0], fwd_hor[0][1], fwd_ver[1][0]])

            else:
                # The rest of the first row
                # Roads to the left are already created
                # position = (0, j) 

                # To the left [0][j], already created

                # Above
                fwd_ver[0][j] = DummyRoad(j, -1, j, 0)
                bwd_ver[0][j] = DummyRoad(j, 0, j, -1)

                # To the right
                fwd_hor[0][j+1] = DummyRoad(j, 0, j+1, 0)
                bwd_hor[0][j+1] = DummyRoad(j+1, 0, j, 0)

                # Below
                fwd_ver[1][j] = DummyRoad(j, 0, j, 1)
                bwd_ver[1][j] = DummyRoad(j, 1, j, 0)

                # Add to junction
                junctions[0][j] = DummyJunction(j, 0, [fwd_hor[0][j], fwd_ver[0][j], bwd_hor[0][j+1], bwd_ver[1][j]], 
                                                    [bwd_hor[0][j], bwd_ver[0][j], fwd_hor[0][j+1], fwd_ver[1][j]])

        elif j == 0:
            # The first column of nodes (except corner)
            # Roads above are already created
            # position = (i, 0)

            # To the left
            fwd_hor[i][0] = DummyRoad(-1, i, 0, i)
            bwd_hor[i][0] = DummyRoad(0, i, -1, i)

            # Above [i][0], already created

            # To the right
            fwd_hor[i][1] = DummyRoad(0, i, 1, i)
            bwd_hor[i][1] = DummyRoad(1, i, 0, i)

            # Below
            fwd_ver[i+1][0] = DummyRoad(0, i, 0, i+1)
            bwd_ver[i+1][0] = DummyRoad(0, i+1, 0, i)

            # Add to junction
            junctions[i][0] = DummyJunction(0, i, [fwd_hor[i][0], fwd_ver[i][0], bwd_hor[i][1], bwd_ver[i+1][0]], 
                                                    [bwd_hor[i][0], bwd_ver[i][0], fwd_hor[i][1], fwd_ver[i+1][0]])
        
        else:
            # node is not at left or upper edge of grid
            # position = (i, j)

            # To the left, already created

            # Above, already created

            # To the right
            fwd_hor[i][j+1] = DummyRoad(j, i, j+1, i)
            bwd_hor[i][j+1] = DummyRoad(j+1, i, j, i)

            # Below
            fwd_ver[i+1][j] = DummyRoad(j, i, j, i+1)
            bwd_ver[i+1][j] = DummyRoad(j, i+1, j, i)

            # Add to junction
            junctions[i][j] = DummyJunction(j, i, [fwd_hor[i][j], fwd_ver[i][j], bwd_hor[i][j+1], bwd_ver[i+1][j]], 
                                                    [bwd_hor[i][j], bwd_ver[i][j], fwd_hor[i][j+1], fwd_ver[i+1][j]])

In [4]:
# for i in range(N):
#     for j in range(M):
#         print("---------------------------")
#         print(f"i = {i}, j = {j}")
#         print("Position of junction:")
#         print(junctions[i][j].x_pos, junctions[i][j].y_pos)
#         print("Position of incoming roads:")
#         for road in junctions[i][j].in_roads:
#             print(f"Start: {road.start_x}, {road.start_y}, end: {road.end_x}, {road.end_y}")
#         print("\nPosition of outgoing roads:")
#         for road in junctions[i][j].out_roads:
#             print(f"Start: {road.start_x}, {road.start_y}, end: {road.end_x}, {road.end_y}")
#         print("---------------------------\n")

#### Checking with actual road network

First step only constructing the network, not caring about densities and so on

In [15]:
n = 10
m = 7
L = 50
N = 5
Vmax = torch.tensor([30.])
control_points = []
distribution = [[0.25, 0.25, 0.25, 0.25],
                [0.25, 0.25, 0.25, 0.25],
                [0.25, 0.25, 0.25, 0.25],
                [0.25, 0.25, 0.25, 0.25]]

fwd_hor = [[None for _ in range(m+1)] for _ in range(n)]
fwd_ver = [[None for _ in range(m)] for _ in range(n+1)]
bwd_hor = [[None for _ in range(m+1)] for _ in range(n)]
bwd_ver = [[None for _ in range(m)] for _ in range(n+1)]

# Creating the junctions
junctions = [[None for _ in range(m)] for _ in range(n)]

for i in range(n): 
    for j in range(m):
        # Node at position (i,j)
        
        if i == 0:
            # The first row of nodes
            if j == 0:
                # The first corner of grid
                # Need to create all 8 roads
                
                # To the left
                fwd_hor[0][0] = rd.Road(2, L, N, torch.tensor([30.0], requires_grad=True), control_points, left_pos=(-1, 0), right_pos=(0, 0),
                                        inflow = 0.2)
                bwd_hor[0][0] = rd.Road(2, L, N, torch.tensor([30.0], requires_grad=True), control_points, left_pos=(0, 0), right_pos=(-1, 0),
                                        inflow = 0.2)

                # Above
                fwd_ver[0][0] = rd.Road(1, L, N, torch.tensor([30.0], requires_grad=True), control_points, left_pos=(0, -1), right_pos=(0, 0),
                                        inflow = 0.2)
                bwd_ver[0][0] = rd.Road(1, L, N, torch.tensor([30.0], requires_grad=True), control_points, left_pos=(0, 0), right_pos=(0, -1),
                                        inflow = 0.2)

                # To the right
                fwd_hor[0][1] = rd.Road(2, L, N, torch.tensor([30.0], requires_grad=True), control_points, left_pos=(0, 0), right_pos=(1, 0),
                                        inflow = 0.2)
                bwd_hor[0][1] = rd.Road(2, L, N, torch.tensor([30.0], requires_grad=True), control_points, left_pos=(1, 0), right_pos=(0, 0),
                                        inflow = 0.2)
            
                # Below
                fwd_ver[1][0] = rd.Road(1, L, N, torch.tensor([30.0], requires_grad=True), control_points, left_pos=(0, 0), right_pos=(0, 1),
                                        inflow = 0.2)
                bwd_ver[1][0] = rd.Road(1, L, N, torch.tensor([30.0], requires_grad=True), control_points, left_pos=(0, 1), right_pos=(0, 0),
                                        inflow = 0.2)

                # Add to junction 
                junctions[0][0] = jn.Junction([fwd_hor[0][0], fwd_ver[0][0], bwd_hor[0][1], bwd_ver[1][0],
                                               fwd_hor[0][1], fwd_ver[1][0], bwd_hor[0][0], bwd_ver[0][0]],
                                               [0,1,2,3], [4,5,6,7], distribution, [], [])

            else:
                # The rest of the first row
                # Roads to the left are already created
                # position = (0, j) 

                # To the left [0][j], already created

                # Above
                fwd_ver[0][j] = rd.Road(1, L, N, torch.tensor([30.0], requires_grad=True), control_points, left_pos=(j, -1), right_pos=(j, 0),
                                        inflow = 0.2)
                bwd_ver[0][j] = rd.Road(1, L, N, torch.tensor([30.0], requires_grad=True), control_points, left_pos=(j, 0), right_pos=(j, -1),
                                        inflow = 0.2)

                # To the right
                fwd_hor[0][j+1] = rd.Road(2, L, N, torch.tensor([30.0], requires_grad=True), control_points, left_pos=(j, 0), right_pos=(j+1, 0),
                                        inflow = 0.2)
                bwd_hor[0][j+1] = rd.Road(2, L, N, torch.tensor([30.0], requires_grad=True), control_points, left_pos=(j+1, 0), right_pos=(j, 0),
                                        inflow = 0.2)

                # Below
                fwd_ver[1][j] = rd.Road(1, L, N, torch.tensor([30.0], requires_grad=True), control_points, left_pos=(j, 0), right_pos=(j, 1),
                                        inflow = 0.2)
                bwd_ver[1][j] = rd.Road(1, L, N, torch.tensor([30.0], requires_grad=True), control_points, left_pos=(j, 1), right_pos=(j, 0),
                                        inflow = 0.2)
                
                # Add to junction
                junctions[0][j] = jn.Junction([fwd_hor[0][j], fwd_ver[0][j], bwd_hor[0][j+1], bwd_ver[1][j],
                                               fwd_hor[0][j+1], fwd_ver[1][j], bwd_hor[0][j], bwd_ver[0][j]],
                                               [0,1,2,3], [4,5,6,7], distribution, [], [])

        elif j == 0:
            # The first column of nodes (except corner)
            # Roads above are already created
            # position = (i, 0)

            # To the left
            fwd_hor[i][0] = rd.Road(2, L, N, torch.tensor([30.0], requires_grad=True), control_points, left_pos=(-1, i), right_pos=(0, i),
                                        inflow = 0.2)
            bwd_hor[i][0] = rd.Road(2, L, N, torch.tensor([30.0], requires_grad=True), control_points, left_pos=(0, i), right_pos=(-1, i),
                                        inflow = 0.2)

            # Above [i][0], already created

            # To the right
            fwd_hor[i][1] = rd.Road(2, L, N, torch.tensor([30.0], requires_grad=True), control_points, left_pos=(0, i), right_pos=(1, i),
                                        inflow = 0.2)
            bwd_hor[i][1] = rd.Road(2, L, N, torch.tensor([30.0], requires_grad=True), control_points, left_pos=(1, i), right_pos=(0, i),
                                        inflow = 0.2)

            # Below
            fwd_ver[i+1][0] = rd.Road(1, L, N, torch.tensor([30.0], requires_grad=True), control_points, left_pos=(0, i), right_pos=(0, i+1),
                                        inflow = 0.2)
            bwd_ver[i+1][0] = rd.Road(1, L, N, torch.tensor([30.0], requires_grad=True), control_points, left_pos=(0, i+1), right_pos=(0, i),
                                        inflow = 0.2)

            # Add to junction
            junctions[i][0] = jn.Junction([fwd_hor[i][0], fwd_ver[i][0], bwd_hor[i][1], bwd_ver[i+1][0],
                                           fwd_hor[i][1], fwd_ver[i+1][0], bwd_hor[i][0], bwd_ver[i][0]],
                                           [0,1,2,3], [4,5,6,7], distribution, [], [])
        
        else:
            # node is not at left or upper edge of grid
            # position = (i, j)

            # To the left, already created

            # Above, already created

            # To the right
            fwd_hor[i][j+1] = rd.Road(2, L, N, torch.tensor([30.0], requires_grad=True), control_points, left_pos=(j, i), right_pos=(j+1, i),
                                        inflow = 0.2)
            bwd_hor[i][j+1] = rd.Road(2, L, N, torch.tensor([30.0], requires_grad=True), control_points, left_pos=(j+1, i), right_pos=(j, i),
                                        inflow = 0.2)

            # Below
            fwd_ver[i+1][j] = rd.Road(1, L, N, torch.tensor([30.0], requires_grad=True), control_points, left_pos=(j, i), right_pos=(j, i+1),
                                        inflow = 0.2)
            bwd_ver[i+1][j] = rd.Road(1, L, N, torch.tensor([30.0], requires_grad=True), control_points, left_pos=(j, i+1), right_pos=(j, i),
                                        inflow = 0.2)

            # Add to junction
            junctions[i][j] = jn.Junction([fwd_hor[i][j], fwd_ver[i][j], bwd_hor[i][j+1], bwd_ver[i+1][j],
                                           fwd_hor[i][j+1], fwd_ver[i+1][j], bwd_hor[i][j], bwd_ver[i][j]],
                                           [0,1,2,3], [4,5,6,7], distribution, [], [])

In [17]:
roads = [road for row in fwd_hor for road in row] + [road for row in fwd_ver for road in row] +\
        [road for row in bwd_hor for road in row] + [road for row in bwd_ver for road in row]

jncs = [junction for row in junctions for junction in row]
# network = nw.Network([road for road in fwd_hor])

In [12]:
network2 = nw.RoadNetwork(roads, jncs, 100.0, optimizing=False)
# print([road.gamma for road in network2.roads])
# print([road.dx for road in network2.roads])
# print([road.max_dt() for road in network2.roads])

In [13]:
network = nw.RoadNetwork(roads, jncs, 100.0, optimizing = False)
dens, que = network.solve_cons_law()
#len(network.junctions)

In [12]:
print(len(dens[0]))

168


In [18]:
print(dens[0].keys())

dict_keys([0, tensor(0.6000, grad_fn=<AddBackward0>), tensor(1.2000, grad_fn=<AddBackward0>), tensor(1.8000, grad_fn=<AddBackward0>), tensor(2.4000, grad_fn=<AddBackward0>), tensor(3.0000, grad_fn=<AddBackward0>), tensor(3.6000, grad_fn=<AddBackward0>), tensor(4.2000, grad_fn=<AddBackward0>), tensor(4.8000, grad_fn=<AddBackward0>), tensor(5.4000, grad_fn=<AddBackward0>), tensor(6.0000, grad_fn=<AddBackward0>), tensor(6.6000, grad_fn=<AddBackward0>), tensor(7.2000, grad_fn=<AddBackward0>), tensor(7.8000, grad_fn=<AddBackward0>), tensor(8.4000, grad_fn=<AddBackward0>), tensor(9., grad_fn=<AddBackward0>), tensor(9.6000, grad_fn=<AddBackward0>), tensor(10.2000, grad_fn=<AddBackward0>), tensor(10.8000, grad_fn=<AddBackward0>), tensor(11.4000, grad_fn=<AddBackward0>), tensor(12.0000, grad_fn=<AddBackward0>), tensor(12.6000, grad_fn=<AddBackward0>), tensor(13.2000, grad_fn=<AddBackward0>), tensor(13.8000, grad_fn=<AddBackward0>), tensor(14.4000, grad_fn=<AddBackward0>), tensor(15.0000, grad_f

In [18]:
def run():
    network = nw.RoadNetwork(roads, jncs, 100, optimizing = False)
    network.solve_cons_law()

cProfile.run('run()','profile_results_full')

In [10]:
prnt
73.4
75.5
73.2


64.4
67.4

137.9

NameError: name 'prnt' is not defined

In [ ]:
def run_optimizing():
    network = nw.RoadNetwork(roads, jncs, 100, optimizing = True)
    network.solve_cons_law()

cProfile.run('run_optimizing()','profile_results_optimizing')

In [ ]:
stats = pstats.Stats('profile_results')
stats.sort_stats('cumtime').print_stats()

Wed Jan 31 20:16:39 2024    profile_results

         22866252 function calls (22866114 primitive calls) in 195.465 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000  196.437  196.437 {built-in method builtins.exec}
        1    0.000    0.000  196.437  196.437 <string>:1(<module>)
        1    0.098    0.098  196.437  196.437 C:\Users\torje\AppData\Local\Temp\ipykernel_27396\3318895114.py:1(run)
        1    0.387    0.387  196.332  196.332 c:\Users\torje\OneDrive - NTNU\Dokumenter\Semester10\kristiansand\notebooks\../src_variable\network.py:214(solve_cons_law)
     7000   19.846    0.003  167.583    0.024 c:\Users\torje\OneDrive - NTNU\Dokumenter\Semester10\kristiansand\notebooks\../src_variable\junction.py:298(apply_bc)
     7000    9.307    0.001  145.521    0.021 c:\Users\torje\OneDrive - NTNU\Dokumenter\Semester10\kristiansand\notebooks\../src_variable\junction.py:154(divide_flux)
     7000

In [ ]:
print(len(roads))
print(len(jncs))
network = nw.RoadNetwork(roads, jncs, 20)
# network.draw_network()
# for road in network.roads:
#     print(road.rho)
#network.solve_cons_law()

314
70
